In [1]:
import pandas as pd
from diskcache import Cache
from pathlib import Path
import requests
import json
import os
import re

Definition et creation des fonctions pour l'embeddings a partir de l'API JDM

Mettre les generiques + les infos semantiques (relation 36) + marqueurs de type (numero de la relation)

Pour le vecteur B regarder l'article (determine ou non determine)

FAIRE DES PRODUITS SCALAIRES SUR DES VECTEURS NORMES


Useful links :

https://www.jeuxdemots.org/jdm-about.php

https://jdm-api.demo.lirmm.fr/schema

https://jdm-api.demo.lirmm.fr/v0/relations/from/{node1_id}

https://jdm-api.demo.lirmm.fr/v0/relations/to/{node2_name}

https://jdm-api.demo.lirmm.fr/v0/node_by_name/{node_name}

In [4]:
CACHE_DIR = "./.jdm_cache"
relations_path = Path("/home/mario/Desktop/portfolio_projects/natural_language_semantics/relations")

In [5]:
cache = Cache()

In [6]:
def load_relations(directory_path: str):

    rows = []

    for filename in os.listdir(directory_path):
        full_path = os.path.join(directory_path, filename)
        relation_num = int(re.split("[-.]",filename)[0])
        if os.path.isfile(full_path):
            
            with open(full_path, "r", encoding="utf-8") as f:
                data = f.read()
                data = re.split("\n",data)
                
                for l in data:
                     rows.append({"content": l, "filename": relation_num})

           

    df = pd.DataFrame(rows)

    print(df.head())
    

    return df

In [7]:
def get_relations_JDM(node_name: str):
    with cache as c:
        if c.__contains__(node_name):
            return c.get(node_name)
        else:
            res = requests.get(f"https://jdm-api.demo.lirmm.fr/v0/relations/to/{node_name}")
            res = json.loads(res.text)
            c.set(node_name,res)
            return res

In [8]:

df = load_relations(relations_path)

                     content  filename
0    Le travail de l’ouvrier         9
1   Le projet de l’ingénieur         9
2  Le discours du professeur         9
3     Le roman de l’écrivain         9
4    Le tableau de l’artiste         9


In [9]:

def remove_article(s):
    ARTICLES = r"(le|la|les|un|une|l')"
    
    if not s:
        return s
    # normaliser apostrophe
    s = s.replace("’", "'").strip()
    # enlever article au début
    s = re.sub(rf"{ARTICLES}\s*", "", s, flags=re.IGNORECASE)
    return s.strip()

In [10]:
df.iloc[:50,0].apply(lambda x: remove_article(x))

0                 travail de ouvrier
1                projet de ingénieur
2             discours du professeur
3                  roman de écrivain
4                   tabau de artiste
5                concert du musicien
6            découverte du chercheur
7                film du réalisateur
8                   pn de architecte
9                     poème du poète
10                  spectacde acteur
11            rapport du journaliste
12            dessin de illustrateur
13        programme de informaticien
14           logiciel du développeur
15               peinture de peintre
16                roman du romancier
17                   texte de auteur
18            chanson du compositeur
19    pièce de théâtre du dramaturge
20             manuscrit de écrivain
21              traité du philosophe
22                projet de étudiant
23            prototype de ingénieur
24                pn du constructeur
25                dessin du designer
26              sculpture de artiste
2

In [11]:
embedding = dict()
embedding["info_sem"] = []
embedding["type_marqueur"] = set()
embedding["hyperonyme"] = []

In [69]:
# obtenir les relations d'un mot
node_name = "avignon"
res = requests.get(f"https://jdm-api.demo.lirmm.fr/v0/relations/to/{node_name}")

In [70]:
# parser le json
res = json.loads(res.text)

In [75]:
"""types =   []
for x in res["relations"]:
    if(not x["type"] in types):
        #print(x)
        types.append(x["type"])

#cache.get(node_name)
types.sort()
print(types)"""

{'id': 2342, 'name': 'Misanthrope', 'type': 1, 'w': 82}


In [72]:
# filtrer les noeuds pour les relations voulues + les poids negatifs
for r in res["relations"]:
    
    
    if(r["w"] <= 0): continue
    
    if(r["type"] == 36):
        embedding["info_sem"].append(node["id"])
    elif(r["type"] == 6):
        embedding["hyperonyme"].append(node["id"])
    
    embedding["type_marqueur"].add(node["type"])

In [55]:
embedding["hyperonyme"]

[1545212,
 1598796,
 1622767,
 1630304,
 1647842,
 1647834,
 1647835,
 1647836,
 1647837,
 1647838,
 1647839,
 1647840,
 1647841,
 1772041,
 1774074,
 1774408,
 1776349,
 1780040,
 1780978,
 1784948,
 1820166,
 1821167,
 1829033,
 1932080,
 1932084,
 1973508,
 1973509,
 1973615,
 1982047,
 1982182,
 2013594,
 2199269,
 2214695,
 2218967,
 2280338,
 2281216,
 2379406,
 2959548,
 3061874,
 3109436,
 3109441,
 3109442,
 3109443,
 3109444,
 3109445,
 3109446,
 3109447,
 3109448,
 3161593,
 3166954,
 3199098,
 8529631,
 8540999,
 9016994,
 9054393,
 9136830,
 9910198,
 9911667,
 9911668,
 9918135,
 9926175,
 10526435,
 10526422,
 10526446,
 10526445,
 10526443,
 10526440,
 10526442,
 10526438,
 10526463,
 10526471,
 10526469,
 10526470,
 10526437,
 10526468,
 10526448,
 10526439,
 10526467,
 10526466,
 10526465,
 10526456,
 10526449,
 10526452,
 10526453,
 10526447,
 10526451,
 10526457,
 10568993,
 10720078,
 12138517,
 12138518,
 12392388,
 12972996,
 12972998,
 13433856,
 13951533,
 1399

In [73]:
# utiliser le nombre de fois que la relation est presente comme poids ?
embedding["type_marqueur"]

{0,
 1,
 2,
 3,
 5,
 6,
 7,
 8,
 9,
 10,
 13,
 14,
 15,
 16,
 19,
 20,
 21,
 22,
 23,
 27,
 28,
 31,
 35,
 41,
 42,
 50,
 51,
 53,
 56,
 59,
 68,
 71,
 75,
 78,
 80,
 82,
 99,
 102,
 115,
 120,
 121,
 129,
 130,
 147,
 153,
 155,
 161,
 163,
 164,
 170,
 174,
 175,
 333,
 666,
 777,
 2000}

In [76]:
embedding["info_sem"]

[]